In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pickle
import pandas as pd
import numpy as np
from joblib import dump, load
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix, classification_report, hamming_loss, zero_one_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval

In [ ]:
THEMES = [5, 6, 26, 33, 139, 163, 232, 313, 339, 350, 406, 409, 555, 589,
          597, 634, 660, 695, 729, 766, 773, 793, 800, 810, 852, 895, 951, 975]
TRAIN_DATA_PATH = '../train.csv'
TEST_DATA_PATH = '../test.csv'
VALIDATION_DATA_PATH = '../validation.csv'

In [ ]:
def groupby_process(df):
    new_df = df.sort_values(['process_id', 'page'])
    new_df = new_df.groupby(
                ['process_id', 'themes'],
                group_keys=False
            ).apply(lambda x: x.body.str.cat(sep=' ')).reset_index()
    new_df = new_df.rename(index=str, columns={0: "body"})
    return new_df

def get_data(path, preds=None, key=None):
    data = pd.read_csv(path)
    data = data.rename(columns={ 'pages': 'page'})
    data = groupby_process(data)
    data.themes = data.themes.apply(lambda x: literal_eval(x))
    return data

def transform_y(train_labels, test_labels):
    mlb = MultiLabelBinarizer()
    mlb.fit(train_labels)

    mlb_train = mlb.transform(train_labels)
    mlb_test = mlb.transform(test_labels)

    print(mlb.classes_)

    return mlb_train, mlb_test, mlb

In [ ]:
train_data = get_data(TRAIN_DATA_PATH)
test_data = get_data(TEST_DATA_PATH)
validation_data = get_data(VALIDATION_DATA_PATH)

train_data.themes = train_data.themes.apply(lambda x: list(set(sorted([i if i in THEMES else 0 for i in x]))))
test_data.themes = test_data.themes.apply(lambda x: list(set(sorted([i if i in THEMES else 0 for i in x]))))
validation_data.themes = validation_data.themes.apply(lambda x: list(set(sorted([i if i in THEMES else 0 for i in x]))))

y_train, y_test, mlb = transform_y(train_data.themes, test_data.themes)

In [ ]:
from xgboost.sklearn import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier

xgboost = OneVsRestClassifier(XGBClassifier(
                n_jobs=-1,
                max_depth=4,
                learning_rate=0.1,
                n_estimators=500,
            ),
            n_jobs=1)

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uni

param_dist = {"estimator__max_depth": sp_randint(1, 8),
              "estimator__learning_rate": [0.1, 0.3, 0.5],
              "estimator__n_estimators": [30, 100, 300, 500, 1000]}


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield([x for x in sentence.split(" ") if len(x) > 1])

def texts_to_feats(texts, model, dictionary):
    texts_words = sent_to_words(texts)
    texts_corpus = [dictionary.doc2bow(text) for text in texts_words]
    texts_topic_dist = [model.get_document_topics(text) for text in text_corpus]
    return np.array([y[1] for y in dist.sort(key=lambda x: x[0])] for dist in texts_topic_dist)

In [ ]:
import gensim.corpora as corpora
import gensim

model_opt = "first"

model = gensim.models.ldamodel.LdaModel.load("models/lda_big_{}".format(model_opt))
dictionary = corpora.Dictionary.load("dicts/big_dict")

In [ ]:
X_train = texts_to_feats(train_data.body, model, dictionary)
X_valid = texts_to_feats(validation_data.body, model, dictionary)
X_test = texts_to_feats(test_data.body, model, dictionary)
y_valid = mlb.transform(validation_data.themes)

In [ ]:
print('X_train: {}, \n\ty_train: {}'.format(X_train.shape, y_train.shape))
print('X_test: {}, \n\ty_test: {}'.format(X_test.shape, y_test.shape))
print('Classes: ', mlb.classes_)
print('We\'re classifying {} themes!'.format(y_train.shape[1]))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(xgboost, param_distributions=param_dist,
                                   n_iter=20, n_jobs=1, iid=False, refit=False,
                                   verbose=2, random_state=42)
random_search.fit(X_valid, y_valid)

In [ ]:
random_search.cv_results_

In [ ]:
best_params = random_search.best_params_; best_params

In [ ]:
xgboost = random_search.estimator.set_params(**best_params); xgboost

In [ ]:
xgboost.fit(X_train, y_train)

In [ ]:
target_names=[str(x) for x in mlb.classes_]

In [ ]:
preds_test = xgboost.predict(X_test)
print(classification_report(y_test, preds_test, target_names=target_names, digits=4))
print(accuracy_score(y_test, preds_test))

In [ ]:
from sklearn.externals import joblib

joblib.dump(xgboost, './models/lda_xgboost.pkl')